## 1. 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

data_columns = train.columns.drop(['ID','y'])

print(f'train shape : {train.shape}')
print(f'test shape : {test.shape}')

## 2. 데이터 전처리

### 이상치 처리

In [ ]:
# # IQR을 사용한 이상치 탐지 및 제거 함수 정의
# def remove_outliers(df, column):
#     # 1사분위수(25%)와 3사분위수(75%) 계산
#     Q1 = df[column].quantile(0.25)
#     Q3 = df[column].quantile(0.75)
#     # IQR 계산
#     IQR = Q3 - Q1
#     # 이상치 범위 설정 (1.5 * IQR보다 더 크거나 작은 값들)
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     # 이상치가 아닌 데이터만 반환
#     return df[(df[column] >= lower_bound)] 
#             #   & (df[column] <= upper_bound)]

# # target 컬럼에서 이상치 제거
# train = remove_outliers(train, 'y')
# print(f'train shape : {train.shape}')

In [ ]:
from scipy import stats

def remove_outliers_zscore(df, column, threshold=3):
    """
    Z-score를 사용하여 이상치를 제거하는 함수

    Parameters:
    df (DataFrame): 데이터 프레임
    column (str): 이상치를 제거할 열 이름
    threshold (float): 이상치로 간주할 Z-score의 임계값 (기본값은 3)

    Returns:
    DataFrame: 이상치가 제거된 데이터 프레임
    """
    # Z-score 계산
    z_scores = stats.zscore(df[column])
    # Z-score가 threshold보다 작은 데이터만 반환
    return df[(z_scores >= -threshold)& (z_scores <= threshold)]

# target 컬럼에서 이상치 제거
train = remove_outliers_zscore(train, 'y')
print(f'train shape : {train.shape}')

### 로그 변환

In [ ]:
# from sklearn.preprocessing import PowerTransformer

# # Yeo-Johnson 변환기 생성
# pt = PowerTransformer(method='yeo-johnson')

# # ID와 y를 제외한 나머지 컬럼에 대해 Yeo-Johnson 변환 수행
# train_yeo_johnson = train.copy()
# train_yeo_johnson[data_columns] = pt.fit_transform(train[data_columns])
# test_yeo_johnson = test.copy()
# test_yeo_johnson[data_columns] = pt.transform(test[data_columns])

### 다중공선성

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA

# 3. 다중공선성 처리 (VIF 계산)
X = train.drop(columns=['ID','y'])
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)
print(vif_data.VIF.sum())

### 시각화

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Create histograms for each variable
# fig, axes = plt.subplots(4, 3, figsize=(20, 20))
# axes = axes.ravel()
 
# for i, col in enumerate(train_yeo_johnson.columns[1:]):  # Skip the ID column
#     sns.histplot(train_yeo_johnson[col], ax=axes[i], kde=True)
#     axes[i].set_title(f'Distribution of {col}')

# plt.tight_layout()
# plt.show()

# # Create box plots for each variable
# fig, axes = plt.subplots(4, 3, figsize=(20, 20))
# axes = axes.ravel()

# for i, col in enumerate(train_yeo_johnson.columns[1:]):  # Skip the ID column
#     sns.boxplot(y=train_yeo_johnson[col], ax=axes[i])
#     axes[i].set_title(f'Box plot of {col}')

# plt.tight_layout()
# plt.show()

In [ ]:
# train_yeo_johnson.head()

### 피쳐 생성

In [ ]:
train['x_1 * x_0'] = train['x_1'] * train['x_0']
train['x_1 + x_0'] = train['x_1'] + train['x_0']


test['x_1 * x_0'] = test['x_1'] * test['x_0']
test['x_1 + x_0'] = test['x_1'] + test['x_0']


train['x_3 * x_4'] = train['x_3'] * train['x_4']
train['x_3 + x_4'] = train['x_3'] + train['x_4']

test['x_3 * x_4'] = test['x_3'] * test['x_4']
test['x_3 + x_4'] = test['x_3'] + test['x_4']


train['x_7 * x_10'] = train['x_7'] * train['x_10']
train['x_7 + x_10'] = train['x_7'] + train['x_10']


test['x_7 * x_10'] = test['x_7'] * test['x_10']
test['x_7 + x_10'] = test['x_7'] + test['x_10']


In [ ]:
train_x = train.drop(columns=['ID','x_5','x_8','y'])
test_x = test.drop(columns=['ID','x_5','x_8']) # 한 개만 제거하는 게 나은 듯
train_y = train['y']

In [ ]:
# train_x = train_yeo_johnson.drop(columns=['ID','x_8','y'])
# test_x = test_yeo_johnson.drop(columns=['ID','x_8'])
# train_y = train_yeo_johnson['y']

### 데이터 스케일링

In [ ]:
# 3. 데이터 스케일링
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
test_x_scaled = scaler.transform(test_x)

## 3. 분석 모델 설계

In [12]:
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from optuna.integration import LightGBMPruningCallback

# 학습/테스트 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(train_x_scaled, train_y, test_size=0.3, random_state=42)

# 목적 함수 정의
def objective(trial):
    # 하이퍼파라미터 샘플링
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-6, 1.0, log=True),  # 축소된 범위
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-6, 1.0, log=True),  # 축소된 범위
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),  # 범위 축소
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 0.9),  # 범위 축소
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 0.9),  # 범위 축소
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 5), 
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50), 
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'force_col_wise': True,
        'random_state' : 42

    }

    # 모델 학습
    model = lgb.LGBMRegressor(**param)
    model.fit(X_train, y_train,
              eval_set=[(X_test, y_test)],
              callbacks=[lgb.early_stopping(100), LightGBMPruningCallback(trial, "rmse")],
              )
    
    # 예측 및 평가
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)
    
    return rmse

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=150)

trial = study.best_trial
best_params = trial.params

C:\Users\jy173\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-08-25 23:42:10,117] Trial 14 finished with value: 1.4656421685637369 and parameters: {'lambda_l1': 1.4335671742489708e-06, 'lambda_l2': 0.04642231481098192, 'num_leaves': 58, 'feature_fraction': 0.509483655967889, 'bagging_fraction': 0.6339034824212495, 'bagging_freq': 4, 'min_child_samples': 22, 'learning_rate': 0.02639541975968309}. Best is trial 13 with value: 1.4638481197550441.


Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 1.46564
[LightGBM] [Warning] feature_fraction is set=0.509483655967889, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.509483655967889
[LightGBM] [Warning] lambda_l1 is set=1.4335671742489708e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4335671742489708e-06
[LightGBM] [Warning] lambda_l2 is set=0.04642231481098192, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.04642231481098192
[LightGBM] [Warning] bagging_fraction is set=0.6339034824212495, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6339034824212495
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.6505060400106166, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6505060400106166
[LightGBM] [Warning] lambda_l1 is set=6.1485923880254345e-06, reg_alpha=0.0 will be 

C:\Users\jy173\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-08-25 23:42:10,330] Trial 15 finished with value: 1.465320759279947 and parameters: {'lambda_l1': 6.1485923880254345e-06, 'lambda_l2': 0.0831458059856633, 'num_leaves': 60, 'feature_fraction': 0.6505060400106166, 'bagging_fraction': 0.6595037713498773, 'bagging_freq': 4, 'min_child_samples': 33, 'learning_rate': 0.09260580116064249}. Best is trial 13 with value: 1.4638481197550441.


Did not meet early stopping. Best iteration is:
[36]	valid_0's rmse: 1.46532
[LightGBM] [Warning] feature_fraction is set=0.6505060400106166, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6505060400106166
[LightGBM] [Warning] lambda_l1 is set=6.1485923880254345e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.1485923880254345e-06
[LightGBM] [Warning] lambda_l2 is set=0.0831458059856633, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0831458059856633
[LightGBM] [Warning] bagging_fraction is set=0.6595037713498773, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6595037713498773
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.834273542751, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.834273542751
[LightGBM] [Warning] lambda_l1 is set=0.00016117476357861574, reg_alpha=0.0 will be ignored. 

C:\Users\jy173\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-08-25 23:42:10,617] Trial 16 finished with value: 1.4663699269621724 and parameters: {'lambda_l1': 0.00016117476357861574, 'lambda_l2': 0.0005376089789406615, 'num_leaves': 58, 'feature_fraction': 0.834273542751, 'bagging_fraction': 0.6535590938556977, 'bagging_freq': 3, 'min_child_samples': 50, 'learning_rate': 0.08954058061532577}. Best is trial 13 with value: 1.4638481197550441.


...

C:\Users\jy173\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-08-25 23:42:26,090] Trial 145 finished with value: 1.4596445775434916 and parameters: {'lambda_l1': 0.1402327768874607, 'lambda_l2': 0.037329544459192066, 'num_leaves': 24, 'feature_fraction': 0.5122150346167821, 'bagging_fraction': 0.8815188802603954, 'bagging_freq': 3, 'min_child_samples': 18, 'learning_rate': 0.099661659800364}. Best is trial 32 with value: 1.4583042466617882.
[I 2024-08-25 23:42:26,204] Trial 146 pruned. Trial was pruned at iteration 50.


Did not meet early stopping. Best iteration is:
[43]	valid_0's rmse: 1.45964
[LightGBM] [Warning] feature_fraction is set=0.5122150346167821, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5122150346167821
[LightGBM] [Warning] lambda_l1 is set=0.1402327768874607, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1402327768874607
[LightGBM] [Warning] lambda_l2 is set=0.037329544459192066, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.037329544459192066
[LightGBM] [Warning] bagging_fraction is set=0.8815188802603954, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8815188802603954
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.5542641002701494, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5542641002701494
[LightGBM] [Warning] lambda_l1 is set=0.486165734850609, reg_alpha=0.0 will be ignored. C

[I 2024-08-25 23:42:26,272] Trial 147 pruned. Trial was pruned at iteration 0.
C:\Users\jy173\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-08-25 23:42:26,443] Trial 148 finished with value: 1.459048949252943 and parameters: {'lambda_l1': 0.5805314442631156, 'lambda_l2': 0.016462425336536184, 'num_leaves': 23, 'feature_fraction': 0.5378508160737393, 'bagging_fraction': 0.8218865076436808, 'bagging_freq': 3, 'min_child_samples': 24, 'learning_rate': 0.0792539263899039}. Best is trial 32 with value: 1.4583042466617882.


[LightGBM] [Warning] feature_fraction is set=0.5378508160737393, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5378508160737393
[LightGBM] [Warning] lambda_l1 is set=0.5805314442631156, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5805314442631156
[LightGBM] [Warning] lambda_l2 is set=0.016462425336536184, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.016462425336536184
[LightGBM] [Warning] bagging_fraction is set=0.8218865076436808, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8218865076436808
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.5378508160737393, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5378508160737393
[LightGBM] [Warning] lambda_l1 is set=0.5805314442631156, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5805314442631156
[LightGBM] [Warning] lambda_l2 is

[I 2024-08-25 23:42:26,503] Trial 149 pruned. Trial was pruned at iteration 0.


In [13]:
# pip install optuna-integration

In [14]:
# 개별 모델 정의
lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=300,random_state=42)
lgbm = lgb.LGBMRegressor(**best_params)
voting_regressor = VotingRegressor(estimators=[('lr', lr), ('rf', rf), ('lgbm', lgbm)],)

In [15]:
# voting_regressor.fit(X_train,y_train)

In [16]:
# # 검증 셋에 대해 예측 수행
# y_pred = voting_regressor.predict(X_test)

# # RMSE 계산
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print(f'검증 셋 RMSE: {rmse}')

## 4. 모델 학습

In [17]:
voting_regressor.fit(train_x_scaled, train_y)

[LightGBM] [Warning] feature_fraction is set=0.5570714486593819, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5570714486593819
[LightGBM] [Warning] lambda_l1 is set=0.0797292301902633, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0797292301902633
[LightGBM] [Warning] lambda_l2 is set=0.025545446612528673, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.025545446612528673
[LightGBM] [Warning] bagging_fraction is set=0.8279494802756857, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8279494802756857
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.5570714486593819, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5570714486593819
[LightGBM] [Warning] lambda_l1 is set=0.0797292301902633, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0797292301902633
[LightGBM] [Warning] lambda_l2 is

VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('rf',
                             RandomForestRegressor(n_estimators=300,
                                                   random_state=42)),
                            ('lgbm',
                             LGBMRegressor(bagging_fraction=0.8279494802756857,
                                           bagging_freq=2,
                                           feature_fraction=0.5570714486593819,
                                           lambda_l1=0.0797292301902633,
                                           lambda_l2=0.025545446612528673,
                                           learning_rate=0.06504688972638147,
                                           min_child_samples=39,
                                           num_leaves=28))])

In [18]:
# import matplotlib.pyplot as plt

# # 모델 학습 후 특성 중요도 추출
# importances = rf.feature_importances_
# indices = np.argsort(importances)[::-1]

# # 특성 중요도 시각화
# plt.figure(figsize=(10, 6))
# plt.title("Feature Importance")
# plt.bar(range(train_x_scaled.shape[1]), importances[indices], align="center")
# plt.xticks(range(train_x_scaled.shape[1]), train_x_scaled.columns[indices], rotation=90)
# plt.tight_layout()
# plt.show()

## 5. 예측값 생성
학습한 모델을 사용하여 예측값을 생성합니다.

In [19]:
y_pred = voting_regressor.predict(test_x_scaled)

[LightGBM] [Warning] feature_fraction is set=0.5570714486593819, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5570714486593819
[LightGBM] [Warning] lambda_l1 is set=0.0797292301902633, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0797292301902633
[LightGBM] [Warning] lambda_l2 is set=0.025545446612528673, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.025545446612528673
[LightGBM] [Warning] bagging_fraction is set=0.8279494802756857, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8279494802756857
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


## 5. 제출 파일 생성
submission 파일을 만들어서 제출합니다.

In [20]:
from datetime import datetime

submission = pd.read_csv('sample_submission.csv')

# 현재 날짜와 시간을 'YYYYMMDD_HHMMSS' 형식으로 포맷팅
today_datetime = datetime.today().strftime('%Y%m%d_%H%M')
submission['y'] = y_pred

# 포맷팅한 날짜와 시간으로 파일 이름 생성
submission.to_csv(f'./submission_{today_datetime}.csv', index=False)